# Gradio Demo: chatbot_multimodal

In [ ]:
!pip install -q gradio 

In [ ]:
# Downloading files from the demo repo
import os
os.mkdir('files')
!wget -q -O files/avatar.png https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/avatar.png
!wget -q -O files/cantina.wav https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/cantina.wav
!wget -q -O files/cheetah.jpg https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/cheetah.jpg
!wget -q -O files/lion.jpg https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/lion.jpg
!wget -q -O files/world.mp4 https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/world.mp4
!wget -q -O files/zebra.jpg https://github.com/gradio-app/gradio/raw/main/demo/chatbot_multimodal/files/zebra.jpg

In [ ]:
import gradio as gr
import os
import plotly.express as px

# Chatbot demo with multimodal input (text, markdown, LaTeX, code blocks, image, audio, & video). Plus shows support for streaming text.

def random_plot():
    df = px.data.iris()
    fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species",
                    size='petal_length', hover_data=['petal_width'])
    return fig

def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)

def add_message(history, message):
    for x in message["files"]:
        history.append(((x,), None))
    if message["text"] is not None:
        history.append((message["text"], None))
    return history, gr.MultimodalTextbox(value=None, interactive=False)

def bot(history):
    history[-1][1] = "Cool!"
    return history

fig = random_plot()

with gr.Blocks(fill_height=True) as demo:
    chatbot = gr.Chatbot(
        [["Image", gr.Image(value=os.path.join(os.path.abspath(''), "files/avatar.png"), render=False)],
         ["Video", gr.Video(value=os.path.join(os.path.abspath(''), "files/world.mp4"), render=False)],
         ["Audio", gr.Audio(value=os.path.join(os.path.abspath(''), "files/cantina.wav"), render=False)],
         ["Plot", gr.Plot(value=fig, render=False)],
         ["Gallery", gr.Gallery(value=[os.path.join(os.path.abspath(''), "files/lion.jpg"),
                                os.path.join(os.path.abspath(''), "files/cheetah.jpg"),
                                os.path.join(os.path.abspath(''), "files/zebra.jpg")], render=False)]],
        elem_id="chatbot",
        bubble_full_width=False,
        scale=1,
    )

    chat_input = gr.MultimodalTextbox(interactive=True, file_types=["image"], placeholder="Enter message or upload file...", show_label=False)

    chat_msg = chat_input.submit(add_message, [chatbot, chat_input], [chatbot, chat_input])
    bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
    bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

    chatbot.like(print_like_dislike, None, None)

demo.queue()
if __name__ == "__main__":
    demo.launch()
